In [28]:
import logging
import pickle
import gensim
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.utils import class_weight
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [6]:
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier

In [12]:
MODEL_PATH = "../models/"
DATA_PATH = "../data/"
train = pd.read_csv(DATA_PATH + "preprocessed/train_ling.csv")

In [15]:
targets = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

In [20]:
features = train[['count_sent', 'count_word', 'count_unique_word', 'count_letters',
       'count_punctuations', 'count_words_upper', 'count_words_title',
       'count_stopwords', 'mean_word_len', 'word_unique_percent',
       'punct_percent', 'count_swear_words']].values

In [21]:
X_train, X_dev, y_train, y_dev = train_test_split(features, targets, test_size=0.3, random_state=42)

In [30]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [33]:
clf_multilabel = OneVsRestClassifier(xgb1)
clf_multilabel.fit(X_train, y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
          n_jobs=1)

In [34]:
pred_dev = clf_multilabel.predict(X_dev)

In [35]:
def mean_roc_auc(y_true, y_pred):
    roc_auc_scores = []
    for i in range(0, y_true.shape[1]):
        roc_auc_scores.append(metrics.roc_auc_score(y_true[:, i], y_pred[:, i]))
    print(roc_auc_scores)
    return np.mean(roc_auc_scores)

In [36]:
mean_roc_auc(y_dev, pred_dev)

[0.7474145619278748, 0.580041178316806, 0.7456208055800402, 0.5109141947377241, 0.6993226656186898, 0.519159484104678]


0.6337454817143021